In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras
import tensorflow as tf


PLOTS_DIR = f"plots/assignment_rnn/"
MODEL_DIR = f"models/assignment_rnn/"
CONFIG_PATH = "../config/workspace_config.yaml"
import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_from_npz(
    data_configs["npz_datapath"]["nominal"], max_events=1000000
)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)
del DataProcessor

2025-11-20 16:56:00.565876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763654160.589230 3329369 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763654160.597003 3329369 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763654160.615269 3329369 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763654160.615291 3329369 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763654160.615293 3329369 computation_placer.cc:177] computation placer alr

In [2]:
TransformerMatcher = Models.FeatureConcatRNN(data_config, name="RNN")

TransformerMatcher.build_model(
    hidden_dim=42,
    num_layers=6,
    dropout_rate=0.1,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)
TransformerMatcher.model.summary()

FeatureConcatRNN is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1763654236.724816 3329369 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


Model: "FeatureConcatRNN"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 6, 8)      │          0 │ flatten_1[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 15)     │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
│                     │                   │            │ repeat_vector_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 6, 42)     │      2,234 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidir_rnn_0         │ (None, 6, 84)     │     28,560 │ jet_embedding[0]… │
│ (Bidirectional)     │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 247,391 (966.38 KB)

 Trainable params: 247,366 (966.27 KB)

 Non-trainable params: 25 (112.00 B)

In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
)

Epoch 1/100


I0000 00:00:1763654257.206670 3331244 service.cc:152] XLA service 0x7f0c1800af60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763654257.206698 3331244 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-20 16:57:37.885836: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-20 16:57:38.270056: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1763654261.102167 3331244 cuda_dnn.cc:529] Loaded cuDNN version 91500


  4/703 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.1303 - loss: 0.0013   

I0000 00:00:1763654272.863233 3331244 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


700/703 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0421 - loss: 0.0013

2025-11-20 16:58:07.035948: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert


703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0421 - loss: 0.0013

2025-11-20 16:58:23.278856: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-20 16:58:26.947857: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert


703/703 ━━━━━━━━━━━━━━━━━━━━ 69s 52ms/step - accuracy: 0.0421 - loss: 0.0013 - val_accuracy: 0.0460 - val_loss: 0.0012
Epoch 2/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0270 - loss: 0.0012 - val_accuracy: 0.0049 - val_loss: 8.6573e-04
Epoch 3/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0215 - loss: 9.0847e-04 - val_accuracy: 0.0584 - val_loss: 8.3840e-04
Epoch 4/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0326 - loss: 8.6941e-04 - val_accuracy: 0.0263 - val_loss: 8.3369e-04
Epoch 5/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0356 - loss: 8.5691e-04 - val_accuracy: 0.0338 - val_loss: 8.3074e-04
Epoch 6/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0354 - loss: 8.5123e-04 - val_accuracy: 0.0437 - val_loss: 8.2860e-04
Epoch 7/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0405 - loss: 8.4367e-04 - val_accuracy: 0.0791 - val_loss: 8.2687e-04
Epoch 8/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/s

In [4]:
TransformerMatcher.save_model(MODEL_DIR + "model.keras")

Model saved to models/assignment_rnn/model.keras
